### 진짜 완전 새롭게 다시 시작

In [1]:
import gymnasium as gym
from horcrux_terrain_v2.envs import PlaneJoyWorld

import ray
from ray.tune.registry import register_env
from ray.rllib.algorithms.sac import SACConfig

### Ray 실행

In [ ]:
import socket
import psutil

# conn_ip = ""
# interfaces = psutil.net_if_addrs()
# for interface_name, addresses in interfaces.items():
#     if "openvpn" in interface_name.lower() and "tap" in interface_name.lower():
#         snicaddrs = interfaces[str(interface_name)]
#         for addrfamily in snicaddrs:
#             if addrfamily.family == socket.AF_INET:
#                 conn_ip = addrfamily.address

# 해당 init을 통해서 VPN을 통한 외부 접속 가능함.
ray.init(dashboard_host="0.0.0.0", dashboard_port=8265)

2025-03-14 20:19:36,670	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://10.130.6.78:8265 


Python version:,3.12.9
Ray version:,2.43.0
Dashboard:,http://10.130.6.78:8265


(SingleAgentEnvRunner pid=69704) 2025-03-14 20:20:40,436	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(SingleAgentEnvRunner pid=69700) 2025-03-14 20:20:46,023	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future! [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_WrappedExecutable pid=69705) Setting up process group for: env:// [rank=0, world_size=1]
(_WrappedExec

In [3]:
env_config = {
    "forward_reward_weight": 6.5,
    "side_cost_weight": 2.0,
    "unhealthy_max_steps": 100,
    "healthy_reward": 0.5,
    "healthy_roll_range": (-35,35),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 0.01,
    "rotation_orientation_cost_weight": 1.2,
    "termination_reward": 0,
    "gait_params": (30, 30, 60, 60, 0),
    "use_friction_chg": True,
    "joy_input_random": True,
}

# JoyWorld
register_env("joy-v1", lambda config: PlaneJoyWorld( forward_reward_weight=env_config["forward_reward_weight"], 
                                                     side_cost_weight=env_config["side_cost_weight"], 
                                                     unhealthy_max_steps=env_config["unhealthy_max_steps"],
                                                     healthy_reward=env_config["healthy_reward"], 
                                                     healthy_roll_range=env_config["healthy_roll_range"],
                                                     terminating_roll_range=env_config["terminating_roll_range"],
                                                     rotation_norm_cost_weight=env_config["rotation_norm_cost_weight"],
                                                     rotation_orientation_cost_weight=env_config["rotation_orientation_cost_weight"],
                                                     termination_reward=env_config["termination_reward"],
                                                     gait_params=env_config["gait_params"],
                                                     use_friction_chg=env_config["use_friction_chg"],
                                                     joy_input_random=env_config["joy_input_random"],
                                                   )
            )


In [5]:
config = (
    SACConfig()
    .environment("joy-v1")
    .env_runners(num_env_runners=16, num_envs_per_env_runner=4)
    .framework('torch')
    .learners(num_learners=1, num_gpus_per_learner=1)
    .training(
        gamma=0.9,
        actor_lr=0.001,
        critic_lr=0.002,
        train_batch_size_per_learner= 32768,

        replay_buffer_config={
            "_enable_replay_buffer_api": True,
            # "type": "EpisodeReplayBuffer",
            "capacity": int(4e7),
            "replay_batch_size": 1024,
            # "replay_sequence_length": 1,
        },

        q_model_config = {
            "fcnet_hiddens": [512, 512, 512, 512, 512, 32],
            "fcnet_activation": "tanh",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "custom_model": None,  # Use this to define custom Q-model(s).
            "custom_model_config": {},
        },
        policy_model_config = {
            "fcnet_hiddens": [512, 512, 512, 512, 512, 32],
            "fcnet_activation": "tanh",
            "post_fcnet_hiddens": [],
            "post_fcnet_activation": None,
            "custom_model": None,  # Use this to define a custom policy model.
            "custom_model_config": {},
        },
    )
)

# Build the SAC algo object from the config and run 1 training iteration.
algo = config.build_algo()

2025-03-14 20:20:08,363	WARNING algorithm_config.py:4703 -- You are running SAC on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-03-14 20:20:08,364	WARNING sac.py:487 -- You are running SAC on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/algorithms/algorithm.py:512: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. Y

### 학습시작

In [6]:
import os
from pprint import pprint

n_iter = 4000
save_iter = 0
save_name = "~/learned_policy/SAC_layer_512_5_32_linear_friction_joy_314"

for i in range(n_iter):
    result = algo.train()
    print(f"{i:03d}", end=", ")
    # result.pop("config")
    # pprint(result)

    if i%40 == 0:
        checkpoint_dir = algo.save(checkpoint_dir=save_name+f"_{save_iter}")
        pprint(f"Checkpoint saved in directory {checkpoint_dir}")
        save_iter += 1

checkpoint_dir = algo.save(checkpoint_dir=save_name+"_final")
pprint(f"Checkpoint saved in directory {checkpoint_dir}")

000, ('Checkpoint saved in directory '
 'TrainingResult(checkpoint=Checkpoint(filesystem=local, '
 'path=~/learned_policy/SAC_layer_512_5_32_linear_friction_joy_314_0), '
 "metrics={'timers': {'training_iteration': 1.7743404020002345, "
 "'restore_env_runners': 1.3715231978022586e-05, 'training_step': "
 "0.07788534697921848, 'env_runner_sampling_timer': 0.06294259019098451, "
 "'replay_buffer_add_data_timer': 0.012423388496532422}, 'env_runners': "
 "{'num_env_steps_sampled_lifetime': 576, 'timers': {'connectors': "
 "{'NumpyToTensor': np.float64(0.00013170512771960998), "
 "'BatchIndividualItems': np.float64(6.939898187275396e-05), "
 "'AddTimeDimToBatchAndZeroPad': np.float64(1.2195371317525106e-05), "
 "'AddObservationsFromEpisodesToBatch': np.float64(3.797402561839562e-05), "
 "'GetActions': np.float64(0.002846431097489402), 'UnBatchToIndividualItems': "
 "np.float64(4.23569471735722e-05), 'ListifyDataForVectorEnv': "
 "np.float64(5.435709172422095e-05), 'NormalizeAndClipActions':

/home/bong/anaconda3/envs/gd/lib/python3.12/site-packages/ray/rllib/utils/replay_buffers/prioritized_episode_buffer.py:475: RuntimeWarning: divide by zero encountered in scalar power
  weight = (p_sample * self.get_num_timesteps()) ** (-beta)


KeyError: 134217727